## Building Blocks
FFT code is made up of small DFT building blocks.

In [1]:
import pyJvsip as pv
from math import pi

In [2]:
def dftCoefE(n):
     m=pv.create('cmview_d',n,n)
     for i in range(n):
         for j in range(n):
             t=(i*j)%n
             x=2.0*pi/n * float(t)
             m[i,j]=complex(cos(x),-sin(x))
     return m

In [3]:
def dftLut(t,n):
    lut=pv.create(t,n)
    w=2.0 * pi / n
    lut.realview.ramp(0,w)
    pv.sin(lut.realview,lut.imagview)
    lut.imagview.neg
    lut.realview.cos
    return lut

In [4]:
def dftBB0(x):
    n=x.length
    if x.type in ['cvview_d','vview_d']:
        t = 'cvview_d'
    elif x.type in ['cvview_f','vview_f']:
        t = 'cvview_f'
    else:
        exit
    y=pv.create(t,x.length).fill(0.0)
    lut=dftLut(t,n)
    for i in range(n):
        for j in range(x.length):
            thta=(i*j) % n
            y[i] += x[j] * lut[thta]
    return y
def dftBB1(x):
    n=x.length
    if x.type in ['cvview_d','vview_d']:
        t = 'cvview_d'
    elif x.type in ['cvview_f','vview_f']:
        t = 'cvview_f'
    else:
        exit
    y=pv.create(t,x.length).fill(0.0)
    lut=dftLut(t,n)
    for i in range(n):
        t=0.0
        for j in range(x.length):
            thta=(i*j) % n
            t+= x[j] * lut[thta]
        y[i]=t
    return y
def dftBB2(x):
    n=x.length
    if x.type in ['cvview_d','vview_d']:
        t = 'cvview_d'
    elif x.type in ['cvview_f','vview_f']:
        t = 'cvview_f'
    else:
        exit
    y=pv.create(t,x.length).fill(0.0)
    lut=dftLut(t,n)
    for j in range(n):
        t=x[j]
        for i in range(x.length):
            thta=(i*j) % n
            y[i] += t * lut[thta]
    return y

In [5]:
z=pv.create('cvview_d',100).randn(5)
z.mprint("%4.3f")

[ 0.617-0.147i -0.109-0.068i  0.021-0.137i  0.514+0.308i  0.801+0.501i -0.102-0.608i -0.718-0.443i -0.242+0.354i  0.182-1.061i  0.050-0.726i  0.664-0.858i -0.623-0.531i -0.261-0.140i -0.216-0.678i -0.377-0.522i  0.585-0.611i -0.323+0.219i  0.829+1.505i  0.567+0.668i -0.949-0.322i -0.750+0.447i  0.908-1.522i  0.430+0.775i  0.351-0.299i -0.906-0.329i -0.062+0.052i -0.243-0.454i -0.078+0.586i  1.053-0.065i -0.261+0.667i  1.275+1.124i -0.484-0.803i -0.491-0.586i  0.233+0.855i -0.025-0.226i  0.151+0.104i  0.207+0.639i  1.147-0.286i  0.690+0.595i -0.434-0.452i  0.586-0.292i -0.780-1.629i  0.170-1.758i  0.171-0.587i -0.396+0.513i -0.100-0.542i  0.666+0.645i -0.494-0.808i  0.072+0.953i  1.553+0.573i  1.003-0.272i -0.812-0.042i  0.474+0.154i  0.162-0.245i  0.158+0.124i -0.632-0.669i -0.698-0.311i  1.533-0.052i  0.344-1.099i -0.318+0.449i -0.119+0.797i -0.002-0.525i -0.419+0.273i -0.294-0.066i -1.303+0.599i  1.613+0.500i -1.222+0.323i -0.092-0.137i  0.227+0.319i -0.130+1.505i  0.310+0.721i  0.09

In [6]:
%%timeit
Z=z.fftop

10000 loops, best of 3: 97.6 µs per loop


In [13]:
Z = z.fftop
Z.mprint("%5.4f")

[ 11.2713-3.0433i -3.0059+0.1030i  4.0063+2.4526i -7.8558+9.6842i -3.4939+7.8698i -7.8666+2.2217i -5.0741+7.4279i  1.8287-10.7265i  2.7870-5.5838i  3.9311-12.1205i -4.7574+3.1582i -0.8201-3.7988i  6.5829-6.1923i -2.7928+4.2443i  4.8919-10.1199i -10.3236-13.3710i -3.1681-0.7968i -1.4950+8.3733i  3.1993+6.2261i  13.9110-7.4186i -6.8505+4.4310i -0.9315+0.1972i -3.6699-0.7540i  4.2380-0.9593i  7.8478+1.5796i -2.3460-2.2259i -5.7385+8.2776i -8.2400-2.5511i  12.0368-7.9904i  6.9861-3.9677i  3.9237+6.8430i  7.0449-0.0490i  2.4706+9.5115i  0.0506-3.3325i  1.3970-3.9247i  3.0118-0.2726i -3.8862-4.2917i  13.3230+7.6638i -3.9932+0.2644i -13.6667+2.4952i  4.7057-3.5383i -3.3860-5.5461i  1.9559+0.2012i  3.6542-7.4617i -1.6865-2.0883i -2.8499+0.9963i  16.0456+4.2971i  0.7982+5.7614i  1.5435-3.2600i -4.0395-3.5765i  4.1749+3.6496i  8.1814-4.5988i  0.8569+13.0625i -2.9710-8.2845i -0.0281-4.2680i -2.6741+9.0621i -5.9769+1.5703i  7.7744-6.9123i  1.4883-1.7450i -3.9282+8.1848i  15.0299+4.1305i  4.9426-1.

In [8]:
zc = z.copy
zc.fftip

In [9]:
(Z-zc).normFro

3.783544127539284e-14

In [10]:
%%timeit
Zbb=dftBB0(z)
print('%.5e'%(Z-Zbb).normFro)

0.00000e+00
0.00000e+00
0.00000e+00
0.00000e+00
1 loop, best of 3: 794 ms per loop


In [11]:
%%timeit
Zbb=dftBB1(z)
print('%.5e'%(Z-Zbb).normFro)

0.00000e+00
0.00000e+00
0.00000e+00
0.00000e+00
1 loop, best of 3: 504 ms per loop


In [12]:
%%timeit
Zbb=dftBB2(z)
print('%.5e'%(Z-Zbb).normFro)

0.00000e+00
0.00000e+00
0.00000e+00
0.00000e+00
1 loop, best of 3: 586 ms per loop
